# Exploring Hacker News Posts
## Introduction
In this project, we will analyze a data set containing submissions to popular technology site [Hacker News](https://news.ycombinator.com/). Hacker News is a site started by startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories known as "posts" are voted and commented upon. Hacker news is very popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

The dataset we will be working with can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts). It has been reduced from from almost 300,000 rows to approximately 20,000 rows by removing submissions that did not receive any comments and randomly sampling from the remaining posts. Below are descriptions of the columns:

| Column | Description |
|--------|-------------|
| `id` | The unique identifier from Hacker News for the post |
| `title` | The title of the post |
| `url` | The URL that the posts links to, if the post has a URL |
| `num_points` | The number of points the post received (upvotes - downvotes) |
| `num_comments` | The number of comments the post received |
| `author` | The name of the account that made the post |
| `created_at` | The date and time the post was made (US Eastern Time) |

Below are the first few rows of the data:

In [1]:
from csv import reader
hn = list(reader(open("HN_posts_year_to_Sep_26_2016.csv", encoding = 'utf8')))
hn_header = hn[0]
hn = hn[1:]
print(hn_header)
for row in hn[0:5]:
    print(row)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26']
['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24']
['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']
['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']
['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14']


In our exploration, we are particularly interested in looking in posts whose titles begin with `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question and `Show HN` posts to show the community a project, product, or just generally something interesting. 

We'll compare these two types of posts to determine the following:
- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at certain times receive more upvotes?

## Cleaning our Data
Since we're only concerned with post titles beginning with `Ask HN` or `Show HN`, we'll create a new list of lists containing just the data for those titles.

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Number of 'Ask HN' posts:", len(ask_posts))
print("Number of 'Show HN' posts:", len(show_posts))
print("Number of other types of posts:", len(other_posts))
print("Total by adding these together:", len(ask_posts) + len(show_posts) + len(other_posts))
print("Number of entries in our Hacker News dataset:", len(hn))

Number of 'Ask HN' posts: 9139
Number of 'Show HN' posts: 10158
Number of other types of posts: 273822
Total by adding these together: 293119
Number of entries in our Hacker News dataset: 293119


Below are the first five rows in the `Ask HN` list:

In [3]:
for row in ask_posts[0:5]:
    print(row)

['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53']
['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17']
['12577908', 'Ask HN: How a DNS problem can be limited to a geographic region?', '', '1', '0', 'kuon', '9/25/2016 22:57']
['12577870', 'Ask HN: Why join a fund when you can be an angel?', '', '1', '3', 'anthony_james', '9/25/2016 22:48']
['12577647', 'Ask HN: Someone uses stock trading as passive income?', '', '5', '2', '00taffe', '9/25/2016 21:50']


Below are the first five rows in the `Show HN` list:

In [4]:
for row in show_posts[0:5]:
    print(row)

['12578335', 'Show HN: Finding puns computationally', 'http://puns.samueltaylor.org/', '2', '0', 'saamm', '9/26/2016 0:36']
['12578182', 'Show HN: A simple library for complicated animations', 'https://christinecha.github.io/choreographer-js/', '1', '0', 'christinecha', '9/26/2016 0:01']
['12578098', 'Show HN: WebGL visualization of DNA sequences', 'http://grondilu.github.io/dna.html', '1', '0', 'grondilu', '9/25/2016 23:44']
['12577991', 'Show HN: Pomodoro-centric, heirarchical project management with ES6 modules', 'https://github.com/jakebian/zeal', '2', '0', 'dbranes', '9/25/2016 23:17']
['12577142', 'Show HN: Jumble  Essays on the go #PaulInYourPocket', 'https://itunes.apple.com/us/app/jumble-find-startup-essay/id1150939197?ls=1&mt=8', '1', '1', 'ryderj', '9/25/2016 20:06']


## Comparing Average Number of Comments
Now, we'd like to determine if `Ask HN` or `Show HN` posts receive more comments on average. 

In [5]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print("Average number of comments in 'Ask HN' posts:", avg_ask_comments)

Average number of comments in 'Ask HN' posts: 10.393478498741656


In [6]:
total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts)
print("Average number of comments in 'Show HN' posts:", avg_show_comments)

Average number of comments in 'Show HN' posts: 4.886099625910612


`Ask HN` posts receive about 10.4 comments per post on average and `Show HN` receive fewer at about 4.9 per post. This is possibly due to the fact that `Ask HN` comments are specifically soliciting responses whereas `Show HN` posts are simply showing the community something of interest.

Since `Ask HN` posts are more likely to receive comments, we'll focus our remaining analysis on just these posts. 

## Analyzing the Times that `Ask HN` Posts Receive the Most Comments
We'll now determine if `Ask HN` posts created at a certain time are more likely to receive comments. We'll complete the following steps to perform this analysis:
1. Calculate the amount of `Ask HN` posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments that `Ask HN` posts receive by the hour they are created. 

In [7]:
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    created_at = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    created_at_hour = created_at.strftime("%H")
    if created_at_hour not in counts_by_hour:
        counts_by_hour[created_at_hour] = 1
        comments_by_hour[created_at_hour] = row[1]
    else:
        counts_by_hour[created_at_hour] += 1
        comments_by_hour[created_at_hour] += row[1]
        
avg_by_hour = []        
for hour in counts_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])
    
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print("Top 5 Hours for 'Ask HN' Post Comments")
template = "{}: {:.2f} average comments per post"
for row in sorted_swap[0:5]:
    format_hour_dt = dt.datetime.strptime(row[1], "%H")
    format_hour_str = format_hour_dt.strftime("%H:%M")
    print(template.format(format_hour_str, row[0]))

Top 5 Hours for 'Ask HN' Post Comments
15:00: 28.68 average comments per post
13:00: 16.32 average comments per post
12:00: 12.38 average comments per post
02:00: 11.14 average comments per post
10:00: 10.68 average comments per post


If your goal is to maximize the number of comments your `Ask HN` posts receive, the five best times to create the post (in Eastern time) are 3 PM, 1 PM, 12 PM, 2 AM, and 10 AM. However, I am located in American Central Time, which is "behind" Eastern by one hour. 

In [8]:
print("Top 5 Hours for 'Ask HN' Post Comments (in American Central Time)")
for row in sorted_swap[0:5]:
    format_hour_dt = dt.datetime.strptime(row[1], "%H")
    central_hour_dt = format_hour_dt - dt.timedelta(hours = 1)
    format_hour_str = central_hour_dt.strftime("%H:%M")
    print(template.format(format_hour_str, row[0]))

Top 5 Hours for 'Ask HN' Post Comments (in American Central Time)
14:00: 28.68 average comments per post
12:00: 16.32 average comments per post
11:00: 12.38 average comments per post
01:00: 11.14 average comments per post
09:00: 10.68 average comments per post
